In [1]:
import pandas as pd
import numpy as np


In [2]:

from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures

In [3]:

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [4]:
df = pd.read_csv('trainx.csv')


In [5]:
df = df.drop(df[df.Views > 3000000].index)


In [6]:
le = LabelEncoder()
df['Tag'] = le.fit_transform(df['Tag'])


In [7]:
df.drop(['ID','Username'], axis=1,inplace =True)
target = df['Upvotes']

In [27]:
from sklearn.preprocessing import Binarizer
bn = Binarizer(threshold=7)
pd_watched = bn.transform([df['Answers']])[0]
df['pd_watched'] = pd_watched

In [28]:
feature_names = [x for x in df.columns if x not in ['Upvotes']]
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(df[feature_names], target,test_size = 0.21,random_state =205)


In [29]:
sc_X = StandardScaler()
x_train = sc_X.fit_transform(x_train)
x_val = sc_X.transform(x_val)

In [30]:
poly_reg = PolynomialFeatures(degree = 4,interaction_only=False, include_bias=True)
X_poly = poly_reg.fit_transform(x_train)
poly_reg.fit(x_train, y_train)
lin_reg_1 = linear_model.LassoLars(alpha=0.021,max_iter=150)
lin_reg_1.fit(X_poly, y_train)

LassoLars(alpha=0.021, copy_X=True, eps=2.220446049250313e-16,
          fit_intercept=True, fit_path=True, max_iter=150, normalize=True,
          positive=False, precompute='auto', verbose=False)

In [31]:

from sklearn.metrics import r2_score

In [32]:
pred_val = lin_reg_1.predict(poly_reg.fit_transform(x_val))

print(r2_score(y_val, pred_val))

0.9034534322501847


In [33]:
test = pd.read_csv('testx.csv')


In [34]:
ids = test['ID']
test.drop(['ID','Username'], axis=1,inplace =True)


In [35]:
le = LabelEncoder()
test['Tag'] = le.fit_transform(test['Tag'])

In [36]:
from sklearn.preprocessing import Binarizer
bn = Binarizer(threshold=7)
pd_watched = bn.transform([test['Answers']])[0]
test['pd_watched'] = pd_watched


In [37]:
test = sc_X.fit_transform(test)
pred_test = lin_reg_1.predict(poly_reg.fit_transform(test))
pred_test=abs(pred_test)

In [38]:
submission = pd.DataFrame({'ID': ids,
                           'Upvotes':pred_test
                           })

submission.to_csv("final_sub05.csv",index=False)